# Proposta de Paralelização segundo a Metodologia de Foster

## Algoritmo Utilizado

O algoritmo escolhido para a resolução do problema Job-Shop é o **Branch and Bound (B&B)**, uma técnica de busca exaustiva com poda que garante a obtenção da solução ótima, ou seja, o menor makespan possível. Este mesmo algoritmo é utilizado tanto na versão sequencial como na versão paralela da aplicação, garantindo consistência nos resultados.

---

## 1. Particionamento

**Objetivo:** Dividir o problema em tarefas menores independentes que possam ser executadas em paralelo.

### Estratégia aplicada:

- O problema original (conjunto de todos os jobs) é replicado localmente para cada thread.
- Cada thread realiza **uma exploração independente** do espaço de soluções usando o mesmo algoritmo de branch and bound.
- Como o espaço de soluções é o mesmo, a esperança é que diferentes threads explorem diferentes caminhos da árvore de busca devido à variação nos tempos de execução ou ordem de chamadas.

### Justificação:

- O algoritmo B&B depende da ordem de exploração da árvore de soluções, e múltiplas instâncias paralelas aumentam a probabilidade de encontrar rapidamente soluções promissoras.
- Cada thread usa **cópias locais** das variáveis de estado (`jobs`, `job_ready`, `machine_available`), evitando interferência direta.

---

## 2. Grupos de Instruções Executadas por Cada Thread

Cada thread executa a seguinte sequência de ações na função `thread_branch`:

```c
void *thread_branch(void *arg)
{
    Job local_jobs[MAX_JOBS];
    int local_job_ready[MAX_JOBS] = {0};
    int local_machine_available[MAX_MACHINES] = {0};

    memcpy(local_jobs, jobs, sizeof(jobs));
    for (int j = 0; j < num_jobs; j++)
        local_jobs[j].next_op = 0;

    branch(local_jobs, local_machine_available, local_job_ready, 0);
    return NULL;
}
```
---


# Padrão de Comunicação entre Threads

## 3. Padrão de Comunicação entre Threads

**Tipo de comunicação:** Comunicação **indireta e mínima**, apenas via memória partilhada protegida por mutex.

### Características:
- Não há troca de mensagens contínua entre threads.
- A única comunicação ocorre quando uma thread encontra uma solução melhor.
- A atualização das variáveis globais (`best_makespan`, `best_schedule`) é feita com proteção de `pthread_mutex`.

---

## Resumo da Proposta (Metodologia de Foster)

| Etapa           | Descrição |
|------------------|-----------|
| **Particionamento** | Múltiplas execuções independentes do algoritmo B&B, com dados locais por thread. |
| **Comunicação**     | Apenas via mutex, protegendo a atualização de `best_makespan` e `best_schedule`. |
| **Aglomeração**     | Cada thread executa a mesma função, simplificando o design. |
| **Mapeamento**      | As threads são geradas com `pthread_create`, e atribuídas ao sistema operativo. |

---

# Análise das Características do Programa Paralelo

## 1. Variáveis Globais Partilhadas

| Variável                          | Tipo de Acesso             | Descrição |
|-----------------------------------|-----------------------------|-----------|
| `jobs[MAX_JOBS]`                 | Leitura                     | Estrutura principal com a sequência de operações e respetivos tempos para cada job. Usada como base para criar cópias locais. |
| `best_makespan`                  | Leitura / Escrita           | Armazena o menor tempo de execução (makespan) encontrado pelas threads. |
| `best_schedule[MAX_JOBS][MAX_OPS]` | Leitura / Escrita         | Guarda os tempos de início das operações para o melhor makespan. |
| `num_jobs`, `num_machines`      | Leitura                     | Utilizadas para percorrer as estruturas de dados em cada thread. |
| `best_mutex`                    | Leitura / Escrita (com sincronização) | Mutex que protege o acesso concorrente às variáveis globais partilhadas. |

---

## 2. Variáveis Locais por Thread

Cada thread trabalha com cópias locais das estruturas e variáveis necessárias para a execução:

| Variável                          | Descrição |
|-----------------------------------|-----------|
| `Job local_jobs[MAX_JOBS]`       | Cópia local dos jobs, incluindo as operações e os tempos de início. |
| `int local_job_ready[MAX_JOBS]`  | Estado de prontidão de cada job para iniciar a próxima operação. |
| `int local_machine_available[MAX_MACHINES]` | Tempo em que cada máquina estará disponível para nova operação. |
| `int id`                          | Identificador da thread, passado como argumento. |

> Estas variáveis são criadas na stack de cada thread, garantindo independência entre execuções paralelas.

---

## 3. Secções Críticas e Condições de Corrida

A única secção crítica do programa ocorre dentro da função `branch()`, no momento da atualização da melhor solução global:

```c
pthread_mutex_lock(&best_mutex);
if (current_makespan < best_makespan) {
    best_makespan = current_makespan;
    for (int j = 0; j < num_jobs; j++)
        for (int o = 0; o < local_jobs[j].num_ops; o++)
            best_schedule[j][o] = local_jobs[j].start_times[o];
}
pthread_mutex_unlock(&best_mutex);
```

### Variáveis envolvidas:
- `best_makespan`
- `best_schedule[][]`

### Problema potencial:
Se múltiplas threads atualizassem essas variáveis sem sincronização, ocorreria uma **condição de corrida**, com possíveis resultados inválidos ou inconsistentes.

---

## 4. Técnica de Exclusão Mútua Utilizada

A correção e o determinismo do programa são garantidos com o uso de `mutex` da biblioteca Pthreads:

```c
pthread_mutex_t best_mutex;
pthread_mutex_init(&best_mutex, NULL);
```

A exclusão mútua é aplicada com:

```c
pthread_mutex_lock(&best_mutex);
// atualização do best_makespan e best_schedule
pthread_mutex_unlock(&best_mutex);
```

> Este mecanismo assegura que apenas uma thread de cada vez possa atualizar as variáveis globais partilhadas, prevenindo conflitos e garantindo consistência dos dados.
